In [1]:
%config IPCompleter.greedy=True


from sklearn.cluster import KMeans, DBSCAN, MeanShift
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from pylab import plot, show
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import silhouette_samples, silhouette_score
import pandas as pd
import numpy as np


In [48]:

arr = np.genfromtxt('../Outputs/Ref/MT-human_clean2_genome_embeddings.csv', delimiter=',')
arr
# df = pd.read_csv('[Query-List of sequences]-MT-human_clean40_genome_embeddings_sigmoid.csv')

array([[ 0.21519931,  0.36393675, -0.13370533, ..., -0.01516642,
        -0.06047211, -0.13087204],
       [ 0.3808372 ,  0.58418429, -0.17480835, ...,  0.01664701,
        -0.35107023, -0.12798193],
       [ 0.48341382,  0.22605367,  0.04467154, ..., -0.04441803,
        -0.19019586, -0.01787082],
       ...,
       [ 0.45108712,  0.45821798, -0.12095217, ..., -0.01665671,
        -0.07261989, -0.29325044],
       [ 0.36358026,  0.45835215, -0.07796567, ...,  0.04284245,
        -0.09766971, -0.02979607],
       [ 0.29455891,  0.51160836, -0.18116045, ..., -0.03222241,
        -0.18442872, -0.06324843]])

In [49]:
# df

In [50]:
# arr = df.values

In [51]:
arr

array([[ 0.21519931,  0.36393675, -0.13370533, ..., -0.01516642,
        -0.06047211, -0.13087204],
       [ 0.3808372 ,  0.58418429, -0.17480835, ...,  0.01664701,
        -0.35107023, -0.12798193],
       [ 0.48341382,  0.22605367,  0.04467154, ..., -0.04441803,
        -0.19019586, -0.01787082],
       ...,
       [ 0.45108712,  0.45821798, -0.12095217, ..., -0.01665671,
        -0.07261989, -0.29325044],
       [ 0.36358026,  0.45835215, -0.07796567, ...,  0.04284245,
        -0.09766971, -0.02979607],
       [ 0.29455891,  0.51160836, -0.18116045, ..., -0.03222241,
        -0.18442872, -0.06324843]])

In [52]:
len(arr)

690

In [53]:
len(arr[0])

1000

In [54]:
def evaluate_kmeans():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(2, 100):
        model = KMeans(n_clusters=i)
        model.fit(arr)
        predicted_labels = model.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        
#         print (i)
#         print (silhouette_avg)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
    return max_score, predicted_labels_final, clusters

In [55]:
def evaluate_gmm():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(2, 100):
        gmm = GaussianMixture(n_components= i, init_params='kmeans')
        gmm = gmm.fit(arr)
        predicted_labels = gmm.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
            
    for i in range(2, 100):
        gmm = GaussianMixture(n_components= i, init_params='random')
        gmm = gmm.fit(arr)
        predicted_labels = gmm.predict(arr)
        silhouette_avg = silhouette_score(arr, predicted_labels)
        if(max_score < silhouette_avg):
            max_score = silhouette_avg
            predicted_labels_final = predicted_labels
            clusters = i
            
    return max_score, predicted_labels_final, clusters

In [56]:
def evaluate_dbscan():
    max_score = -1
    predicted_labels_final = []
    clusters = 0
    for i in range(1, 10000, 1):
        for j in range(50, 200, 10):
            eps = i*0.5
            dbscan = DBSCAN(eps=i,min_samples=j)
            dbscan.fit(arr)
            predicted_labels = dbscan.labels_
#             print (eps)
#             print (j)
#             print (predicted_labels)
            
            labels, count = np.unique(predicted_labels, return_counts= True)
            if (len(labels) == 1):
                continue
            silhouette_avg = silhouette_score(arr, predicted_labels)
            silhouette_avg
            if(max_score < silhouette_avg):
                max_score = silhouette_avg
                predicted_labels_final = predicted_labels
                clusters = labels
                
    return max_score, predicted_labels_final, clusters

In [ ]:
score_kmeans, predicted_label_kmeans, cluster_kmeans = evaluate_kmeans()
score_gmm, predicted_label_gmm, cluster_gmm = evaluate_gmm()
score_dbscan, predicted_label_dbscan, clusters_dbscan = evaluate_dbscan()

In [25]:
 print ('KMeans results: ')
print ('Silhoeutte Score :', score_kmeans)
print ('Number of clusters formed: ', cluster_kmeans)


KMeans results: 
Silhoeutte Score : 0.7215250882219149
Number of clusters formed:  84


###### predicted_labels_Gmm

In [26]:
print ('GMM results: ')
print ('Silhoeutte Score :', score_gmm)
print ('Number of clusters formed: ', cluster_gmm)

GMM results: 
Silhoeutte Score : 0.7241381231835629
Number of clusters formed:  87


In [27]:
print ('DBSCAN results: ')
print ('Silhoeutte Score :', score_dbscan)
print ('Number of clusters formed: ', clusters_dbscan)

DBSCAN results: 
Silhoeutte Score : 0.3699004254163586
Number of clusters formed:  [-1  0]


In [43]:
def findGroups(results, size):
    res = [None]*(size+1)
    
    for i in range(0, len(results)):
        val = results[i]
        if(val == -1):
            continue
        if(res[val] == None):
            res[val] = [i]
        else:
            res[val].append(i)
        
    return res

In [44]:
groups_kmeans = findGroups(predicted_label_kmeans, cluster_kmeans)
groups_gmm = findGroups(predicted_label_gmm, cluster_gmm)
groups_dbcan = findGroups(predicted_label_dbscan, len(clusters_dbscan))

In [45]:
print ('KMM Groups: ')
print(groups_kmeans)

KMM Groups: 
[[115], [135], [24, 74, 114], [131], [37, 87], [48, 98], [44, 94], [7, 57], [49, 99], [43, 93], [17, 67], [22, 72], [136], [20, 70], [5, 55], [19, 69], [45, 95], [108], [27, 77], [106], [12, 62], [1, 51], [14, 64], [0, 50], [31, 81], [39, 89], [42, 92], [3, 53], [119], [120], [38, 88], [130, 132], [30, 80], [4, 54], [139], [9, 59], [28, 78], [13, 63], [26, 76], [23, 73], [107], [10, 60], [36, 86], [29, 79], [41, 91], [25, 75], [126], [40, 90], [35, 85], [110], [8, 58], [33, 83], [18, 68], [124], [11, 61], [133], [21, 71], [46, 96], [34, 84], [15, 65], [47, 97], [112], [6, 56], [113, 125], [32, 82], [2, 52], [116], [101, 118], [109], [111], [16, 66, 102], [105], [104], [122], [129], [134], [100, 128], [121], [137], [117], [123], [138], [127], [103], None]


In [46]:
print('GMM Groups: ')
print(groups_gmm)

GMM Groups: 
[[139], [118], [47, 97], [113, 125], [5, 55], [48, 98], [44, 94], [13, 63], [131], [27, 77], [45, 95], [43, 93], [29, 79], [10, 60], [30, 80], [7, 57], [20, 70], [42, 92], [28, 78], [4, 54], [39, 89], [46, 96], [106], [19, 69], [3, 53], [1, 51], [21, 71], [114], [14, 64], [12, 62], [34, 84], [119], [108], [130, 132], [26, 76], [120], [9, 59], [102], [110], [38, 88], [31, 81], [123], [107], [23, 73], [35, 85], [25, 75], [36, 86], [136], [2, 52], [33, 83], [8, 58], [41, 91], [22, 72], [124], [6, 56], [15, 65], [37, 87], [109], [112], [133], [0, 50], [32, 82], [40, 90], [126], [122], [17, 67], [11, 61], [121], [116], [104], [129], [111], [18, 68], [105], [115], [134], [135], [100, 128], [103], [137], [127], [16, 66], [117], [49, 99], [24, 74], [138], [101], None]


In [47]:
print('DBSCAN Groups: ')
print (groups_dbcan)

DBSCAN Groups: 
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139], None, None]
